In [1]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from quantum.Quantum import QuantumRegressor

os.environ["OMP_NUM_THREADS"] = "12"

In [2]:
# Global variables
OPTIMIZER = None
SHOTS = None
X_DIM = None
BACKEND = None
DEVICE = None
SCALE_FACTORS = None
ANSATZ = None
ENCODER = None
POSTPROCESS = None
ERROR_MITIGATION = None
LAYERS = None
TOKEN = None
HYPERPARAMETERS = None
RE_UPLOAD_DEPTH = None
MAX_ITER = None
TOLERANCE = None
NUM_QUBITS = None
BATCH_SIZE = None
NUM_CORES = None

In [3]:
def parse_settings(settings_file):
    with open(settings_file, 'r') as fp:
        settings = json.load(fp)

    global OPTIMIZER
    OPTIMIZER = settings['OPTIMIZER']

    global SHOTS
    SHOTS = settings['SHOTS']
                
    global BACKEND
    BACKEND = settings['BACKEND']

    global DEVICE
    DEVICE = settings['DEVICE']

    global SCALE_FACTORS
    SCALE_FACTORS = settings['SCALE_FACTORS']

    global POSTPROCESS
    POSTPROCESS = settings['POSTPROCESS']

    global ERROR_MITIGATION
    ERROR_MITIGATION = settings['ERROR_MITIGATION']

    global LAYERS
    LAYERS = settings['LAYERS']

    global HYPERPARAMETERS
    HYPERPARAMETERS = settings['HYPERPARAMETERS']
    # f was removed from HYPERPARAMETERS, this ensures old settings files can still run.
    if 'f' in HYPERPARAMETERS.keys():
        _ = HYPERPARAMETERS.pop('f', None)

    global RE_UPLOAD_DEPTH
    RE_UPLOAD_DEPTH = settings['RE-UPLOAD_DEPTH']

    global MAX_ITER
    MAX_ITER = settings['MAX_ITER']

    global TOLERANCE
    try:
        TOLERANCE = settings['TOLERANCE']
    except KeyError:
        TOLERANCE = None

    global NUM_QUBITS
    try:
        NUM_QUBITS = settings['NUM_QUBITS']
    except KeyError:
        NUM_QUBITS = None

    # classes aren't JSON serializable, so we store the key in the settings file and access it here.
    global ANSATZ
    ANSATZ = ANSATZ_LIST[settings['ANSATZ']]

    global ENCODER
    ENCODER = ENCODER_LIST[settings['ENCODER']]

    global BATCH_SIZE
    BATCH_SIZE = settings['BATCH_SIZE']
    
    global NUM_CORES
    NUM_CORES = settings['NUM_CORES']

In [4]:
def load_dataset(file):
    print(f'Loading dataset from {file}... ')
    data = joblib.load(file)
    X = data['X']
    y = data['y']

    global X_DIM
    _, X_DIM = X.shape
    print(f'Successfully loaded {file} into X and y data. ')
    return X, y

In [5]:
def create_kwargs():
    #  First have to apply specific ansatz settings: setting number of layers and the number of wires based on features
    ANSATZ.layers = LAYERS
    ANSATZ.set_wires(range(X_DIM))

    kwargs = {
        'encoder': ENCODER,
        'variational': ANSATZ,
        'num_qubits': X_DIM,
        'optimizer': OPTIMIZER,
        # 'optimizer': "BFGS",
        'max_iterations': MAX_ITER,
        'tol': TOLERANCE,
        'device': DEVICE,
        'shots': SHOTS,
        'backend': BACKEND,
        'postprocess': POSTPROCESS,
        'error_mitigation': ERROR_MITIGATION,
        'token': TOKEN,
        're_upload_depth': RE_UPLOAD_DEPTH,
        'batch_size': BATCH_SIZE,
        'njobs':NUM_CORES
    }
    return kwargs


In [6]:
settings="./IQP_Full-Pauli-CRZ/IQP_Full-Pauli-CRZ.json"
train_set="../function-calc-test/linear/linear_train.bin"
test_set="../function-calc-test/linear/linear_test.bin"
scaler="../function-calc-test/linear/linear_scaler.bin"

In [7]:
X_train, y_train = load_dataset(train_set)
parse_settings(settings)
if DEVICE == 'qiskit.ibmq':
    save_token(instance, token)

global NUM_QUBITS
global X_DIM

if NUM_QUBITS is not None:
    X_DIM = NUM_QUBITS
elif X_DIM == 1:  # if X_DIM is None and num_qubits wasn't specified anywhere use a default value of 2.
    NUM_QUBITS = 2
    X_DIM = NUM_QUBITS

kwargs = create_kwargs()
title=False
if title is None:
    title = os.path.basename(settings)
    title, _ = os.path.splitext(title)
    
save_circuits=False
if save_circuits:
    plot_circuits(title)

if test_set is not None:
    X_test, y_test = load_dataset(test_set)
else:
    X_test, y_test = None, None

scaler = joblib.load(scaler)

Loading dataset from ../function-calc-test/linear/linear_train.bin... 
Successfully loaded ../function-calc-test/linear/linear_train.bin into X and y data. 
Loading dataset from ../function-calc-test/linear/linear_test.bin... 
Successfully loaded ../function-calc-test/linear/linear_test.bin into X and y data. 


/Users/grierjones/miniconda3/envs/simple_qml/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
model = QuantumRegressor(**kwargs)
# model.fit(X_train, y_train)

12
12


In [9]:
DEVICE

'qulacs.simulator'

In [23]:
dev = qml.device(DEVICE,wires=NUM_QUBITS)
@qml.qnode(dev)
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(NUM_QUBITS))
        variational(params, wires=range(NUM_QUBITS))

    return qml.expval(qml.PauliZ(0))

In [24]:
encoder = ENCODER
variational= ANSATZ

LAYERS=1
re_upload_depth=1
variational.layers = LAYERS
variational.set_wires(range(NUM_QUBITS))


num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
feat = generator.uniform(-1, 1, NUM_QUBITS)

circuit(feat,initial_parameters)

<function __main__.circuit(features, parameters)>

In [15]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
from qiskit.primitives import StatevectorEstimator

qasmcircuit = QuantumCircuit.from_qasm_str(circuit.qtape.to_openqasm())

In [18]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit.quantum_info import Pauli, SparsePauliOp



# Load QASM into a QuantumCircuit
qc = QuantumCircuit.from_qasm_str(circuit.qtape.to_openqasm())
qc.draw('mpl')
# Add measurement of the expectation value of Pauli-Z on the first qubit

# Pauli-Z on the first qubit, Identity on all others
num_qubits = 127
pauli_string = "Z" + "I" * (num_qubits - 1)
observable = SparsePauliOp(pauli_string)
# Define the FakeQuebec backend
# Or save your credentials on disk.
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='pinq-quebec-hub/univ-toronto/default'
)

backend = service.backend(name= 'ibm_quebec')
# backend =FakeQuebec()
# Transpile for the backend
qc_transpiled = transpile(qc, backend=backend)

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(qc_transpiled,observable)])
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

<RuntimeJobV2('cwz5b9n60bqg008q5z0g', 'estimator')>

In [27]:
state="""OPENQASM 2.0;
include "qelib1.inc";
qreg q[16];
creg c[16];
h q[0];
rz(0.7709177971212526) q[0];
h q[1];
rz(-0.2863567564794294) q[1];
h q[2];
rz(0.4556280727424953) q[2];
h q[3];
rz(0.5459386717867756) q[3];
h q[4];
rz(0.17669089675761684) q[4];
h q[5];
rz(-0.28730772139889593) q[5];
h q[6];
rz(0.3881386416186652) q[6];
h q[7];
rz(0.23098675354619957) q[7];
h q[8];
rz(-0.11756691220885163) q[8];
h q[9];
rz(0.8542369374706567) q[9];
h q[10];
rz(-0.7167499738220522) q[10];
h q[11];
rz(0.47911148673168125) q[11];
h q[12];
rz(-0.16901996127639096) q[12];
h q[13];
rz(0.34374085032134305) q[13];
h q[14];
rz(-0.9307508893097289) q[14];
h q[15];
rz(-0.7302874499377254) q[15];
cx q[1],q[0];
rz(-0.2207575198959087) q[0];
cx q[1],q[0];
cx q[2],q[0];
rz(0.3512517901452463) q[0];
cx q[2],q[0];
cx q[3],q[0];
rz(0.42087383821716357) q[0];
cx q[3],q[0];
cx q[4],q[0];
rz(0.13621415689976066) q[0];
cx q[4],q[0];
cx q[5],q[0];
rz(-0.22149063567676344) q[0];
cx q[5],q[0];
cx q[6],q[0];
rz(0.2992229865742967) q[0];
cx q[6],q[0];
cx q[7],q[0];
rz(0.17807179920802588) q[0];
cx q[7],q[0];
cx q[8],q[0];
rz(-0.0906344249743956) q[0];
cx q[8],q[0];
cx q[9],q[0];
rz(0.6585464580544839) q[0];
cx q[9],q[0];
cx q[10],q[0];
rz(-0.5525553109056119) q[0];
cx q[10],q[0];
cx q[11],q[0];
rz(0.36935557192667595) q[0];
cx q[11],q[0];
cx q[12],q[0];
rz(-0.13030049621671475) q[0];
cx q[12],q[0];
cx q[13],q[0];
rz(0.264995939110316) q[0];
cx q[13],q[0];
cx q[14],q[0];
rz(-0.717532425255303) q[0];
cx q[14],q[0];
cx q[15],q[0];
rz(-0.5629915921712884) q[0];
cx q[15],q[0];
cx q[2],q[1];
rz(-0.13047217707151446) q[1];
cx q[2],q[1];
cx q[3],q[1];
rz(-0.15633322728954882) q[1];
cx q[3],q[1];
cx q[4],q[1];
rz(-0.05059663209495288) q[1];
cx q[4],q[1];
cx q[5],q[1];
rz(0.08227250721128339) q[1];
cx q[5],q[1];
cx q[6],q[1];
rz(-0.11114612247825262) q[1];
cx q[6],q[1];
cx q[7],q[1];
rz(-0.06614461753520304) q[1];
cx q[7],q[1];
cx q[8],q[1];
rz(0.033666079649428575) q[1];
cx q[8],q[1];
cx q[9],q[1];
rz(-0.24461651867901837) q[1];
cx q[9],q[1];
cx q[10],q[1];
rz(0.2052461977103988) q[1];
cx q[10],q[1];
cx q[11],q[1];
rz(-0.1371968113325214) q[1];
cx q[11],q[1];
cx q[12],q[1];
rz(0.04840000789138607) q[1];
cx q[12],q[1];
cx q[13],q[1];
rz(-0.09843251496750081) q[1];
cx q[13],q[1];
cx q[14],q[1];
rz(0.26652680575307836) q[1];
cx q[14],q[1];
cx q[15],q[1];
rz(0.2091227454618007) q[1];
cx q[15],q[1];
cx q[3],q[2];
rz(0.24874498486180624) q[2];
cx q[3],q[2];
cx q[4],q[2];
rz(0.08050533276081617) q[2];
cx q[4],q[2];
cx q[5],q[2];
rz(-0.1309054633850167) q[2];
cx q[5],q[2];
cx q[6],q[2];
rz(0.1768468612376025) q[2];
cx q[6],q[2];
cx q[7],q[2];
rz(0.10524404934730065) q[2];
cx q[7],q[2];
cx q[8],q[2];
rz(-0.0535667856280052) q[2];
cx q[8],q[2];
cx q[9],q[2];
rz(0.38921432948520673) q[2];
cx q[9],q[2];
cx q[10],q[2];
rz(-0.3265714092107756) q[2];
cx q[10],q[2];
cx q[11],q[2];
rz(0.21829664332834753) q[2];
cx q[11],q[2];
cx q[12],q[2];
rz(-0.07701023921137319) q[2];
cx q[12],q[2];
cx q[13],q[2];
rz(0.15661798115478007) q[2];
cx q[13],q[2];
cx q[14],q[2];
rz(-0.42407623389955534) q[2];
cx q[14],q[2];
cx q[15],q[2];
rz(-0.33273946336315735) q[2];
cx q[15],q[2];
cx q[4],q[3];
rz(0.09646239349266764) q[3];
cx q[4],q[3];
cx q[5],q[3];
rz(-0.1568523958145982) q[3];
cx q[5],q[3];
cx q[6],q[3];
rz(0.21189989447441737) q[3];
cx q[6],q[3];
cx q[7],q[3];
rz(0.12610460143135147) q[3];
cx q[7],q[3];
cx q[8],q[3];
rz(-0.0641843238973729) q[3];
cx q[8],q[3];
cx q[9],q[3];
rz(0.46636097903393314) q[3];
cx q[9],q[3];
cx q[10],q[3];
rz(-0.39130152871161733) q[3];
cx q[10],q[3];
cx q[11],q[3];
rz(0.2615654887040814) q[3];
cx q[11],q[3];
cx q[12],q[3];
rz(-0.09227453316468512) q[3];
cx q[12],q[3];
cx q[13],q[3];
rz(0.18766142326329086) q[3];
cx q[13],q[3];
cx q[14],q[3];
rz(-0.5081329042741136) q[3];
cx q[14],q[3];
cx q[15],q[3];
rz(-0.3986921604415532) q[3];
cx q[15],q[3];
cx q[5],q[4];
rz(-0.05076465893935846) q[4];
cx q[5],q[4];
cx q[6],q[4];
rz(0.06858056465388522) q[4];
cx q[6],q[4];
cx q[7],q[4];
rz(0.04081325662320864) q[4];
cx q[7],q[4];
cx q[8],q[4];
rz(-0.020773003147206006) q[4];
cx q[8],q[4];
cx q[9],q[4];
rz(0.1509358905251706) q[4];
cx q[9],q[4];
cx q[10],q[4];
rz(-0.1266431956256168) q[4];
cx q[10],q[4];
cx q[11],q[4];
rz(0.0846546382374958) q[4];
cx q[11],q[4];
cx q[12],q[4];
rz(-0.029864288527863193) q[4];
cx q[12],q[4];
cx q[13],q[4];
rz(0.06073587909550385) q[4];
cx q[13],q[4];
cx q[14],q[4];
rz(-0.1644552092900854) q[4];
cx q[14],q[4];
cx q[15],q[4];
rz(-0.12903514442032993) q[4];
cx q[15],q[4];
cx q[6],q[5];
rz(-0.11151522871032137) q[5];
cx q[6],q[5];
cx q[7],q[5];
rz(-0.06636427783468694) q[5];
cx q[7],q[5];
cx q[8],q[5];
rz(0.0337778816586292) q[5];
cx q[8],q[5];
cx q[9],q[5];
rz(-0.2454288680394655) q[5];
cx q[9],q[5];
cx q[10],q[5];
rz(0.2059278017915321) q[5];
cx q[10],q[5];
cx q[11],q[5];
rz(-0.1376524295489167) q[5];
cx q[11],q[5];
cx q[12],q[5];
rz(0.048560739945249516) q[5];
cx q[12],q[5];
cx q[13],q[5];
rz(-0.09875940045754401) q[5];
cx q[13],q[5];
cx q[14],q[5];
rz(0.26741191719757423) q[5];
cx q[14],q[5];
cx q[15],q[5];
rz(0.20981722320781818) q[5];
cx q[15],q[5];
cx q[7],q[6];
rz(0.0896548847533273) q[6];
cx q[7],q[6];
cx q[8],q[6];
rz(-0.04563226160404453) q[6];
cx q[8],q[6];
cx q[9],q[6];
rz(0.3315623645303493) q[6];
cx q[9],q[6];
cx q[10],q[6];
rz(-0.27819836121950514) q[6];
cx q[10],q[6];
cx q[11],q[6];
rz(0.1859616816439339) q[6];
cx q[11],q[6];
cx q[12],q[6];
rz(-0.06560317817625778) q[6];
cx q[12],q[6];
cx q[13],q[6];
rz(0.133419106712571) q[6];
cx q[13],q[6];
cx q[14],q[6];
rz(-0.3612603858620428) q[6];
cx q[14],q[6];
cx q[15],q[6];
rz(-0.2834527788099877) q[6];
cx q[15],q[6];
cx q[8],q[7];
rz(-0.02715639937557369) q[7];
cx q[8],q[7];
cx q[9],q[7];
rz(0.19731741694559488) q[7];
cx q[9],q[7];
cx q[10],q[7];
rz(-0.16555974955747937) q[7];
cx q[10],q[7];
cx q[11],q[7];
rz(0.11066840690684412) q[7];
cx q[11],q[7];
cx q[12],q[7];
rz(-0.039041372139737913) q[7];
cx q[12],q[7];
cx q[13],q[7];
rz(0.07939958307693715) q[7];
cx q[13],q[7];
cx q[14],q[7];
rz(-0.21499112628189243) q[7];
cx q[14],q[7];
cx q[15],q[7];
rz(-0.16868672721664793) q[7];
cx q[15],q[7];
cx q[9],q[8];
rz(-0.10042999903317097) q[8];
cx q[9],q[8];
cx q[10],q[8];
rz(0.0842660812480339) q[8];
cx q[10],q[8];
cx q[11],q[8];
rz(-0.05632765809883595) q[8];
cx q[11],q[8];
cx q[12],q[8];
rz(0.019871154948924958) q[8];
cx q[12],q[8];
cx q[13],q[8];
rz(-0.04041255037232534) q[8];
cx q[13],q[8];
cx q[14],q[8];
rz(0.10942550809178749) q[8];
cx q[14],q[8];
cx q[15],q[8];
rz(0.08585764051405469) q[8];
cx q[15],q[8];
cx q[10],q[9];
rz(-0.6122743025699232) q[9];
cx q[10],q[9];
cx q[11],q[9];
rz(0.40927472913268453) q[9];
cx q[11],q[9];
cx q[12],q[9];
rz(-0.1443830940921532) q[9];
cx q[12],q[9];
cx q[13],q[9];
rz(0.29363613126206345) q[9];
cx q[13],q[9];
cx q[14],q[9];
rz(-0.795081789232033) q[9];
cx q[14],q[9];
cx q[15],q[9];
rz(-0.6238385147080581) q[9];
cx q[15],q[9];
cx q[11],q[10];
rz(-0.343403145572777) q[10];
cx q[11],q[10];
cx q[12],q[10];
rz(0.12114505282025749) q[10];
cx q[12],q[10];
cx q[13],q[10];
rz(-0.2463762454693926) q[10];
cx q[13],q[10];
cx q[14],q[10];
rz(0.6671156755476) q[10];
cx q[14],q[10];
cx q[15],q[10];
rz(0.523433510625438) q[10];
cx q[15],q[10];
cx q[12],q[11];
rz(-0.08097940493446287) q[11];
cx q[12],q[11];
cx q[13],q[11];
rz(0.16469018984787098) q[11];
cx q[13],q[11];
cx q[14],q[11];
rz(-0.4459334423540187) q[11];
cx q[14],q[11];
cx q[15],q[11];
rz(-0.34988910588115185) q[11];
cx q[15],q[11];
cx q[13],q[12];
rz(-0.058099065210427105) q[12];
cx q[13],q[12];
cx q[14],q[12];
rz(0.15731547926909684) q[12];
cx q[14],q[12];
cx q[15],q[12];
rz(0.12343315650910866) q[12];
cx q[15],q[12];
cx q[14],q[13];
rz(-0.3199371021286725) q[13];
cx q[14],q[13];
cx q[15],q[13];
rz(-0.251029629020599) q[13];
cx q[15],q[13];
cx q[15],q[14];
rz(0.6797156934812721) q[14];
cx q[15],q[14];
rx(-2.9031834478991567) q[0];
rz(2.2374346351681274) q[0];
rx(-2.124279635802074) q[1];
rz(-0.11653102652877445) q[1];
rx(0.5538870790868908) q[2];
rz(-2.7701089732198616) q[2];
rx(3.0685849823563265) q[3];
rz(2.1896014521146014) q[3];
rx(1.185519984811111) q[4];
rz(-1.0664830848260962) q[4];
rx(0.6027151026221458) q[5];
rz(1.1443244512903483) q[5];
rx(1.310298985165673) q[6];
rz(-1.8735468004102718) q[6];
rx(0.7296508035125062) q[7];
rz(2.8857841930532784) q[7];
rx(-0.5453433498411107) q[8];
rz(-0.4752248517085489) q[8];
rx(-2.0265424047200344) q[9];
rz(0.7289736950824408) q[9];
rx(1.6051266367087766) q[10];
rz(2.8307710704948708) q[10];
rx(-1.2645670963639735) q[11];
rz(-0.2513610454834194) q[11];
rx(-2.3927921790549602) q[12];
rz(-2.2730977433599113) q[12];
rx(3.1333715483785545) q[13];
rz(2.548569583039808) q[13];
rx(-0.6755078731717168) q[14];
rz(-2.6900220166106816) q[14];
rx(1.5170162304695385) q[15];
rz(-0.046472283379316526) q[15];
crz(-2.342713704951999) q[1],q[0];
crz(2.040432139314733) q[3],q[2];
crz(1.15216312015903) q[5],q[4];
crz(-0.6505549851827448) q[7],q[6];
crz(1.0446536893696319) q[9],q[8];
crz(-1.9205601572028648) q[11],q[10];
crz(-0.32067987718317914) q[13],q[12];
crz(2.3222603299941804) q[15],q[14];
rx(-0.20585092194186005) q[0];
rz(0.5663743333400761) q[0];
rx(2.6898313132619958) q[1];
rz(1.0727336153351201) q[1];
rx(2.581028666086845) q[2];
rz(2.9425403210144534) q[2];
rx(2.7651459044837106) q[3];
rz(-2.9457857080100873) q[3];
rx(0.3668405893501663) q[4];
rz(-0.6572590696008174) q[4];
rx(1.6515908631886775) q[5];
rz(2.680421416956018) q[5];
rx(-0.25194954426493554) q[6];
rz(-2.4580790630331197) q[6];
rx(2.6357052477784872) q[7];
rz(-3.0793042942299422) q[7];
rx(0.7912972428365177) q[8];
rz(-0.7590177064980024) q[8];
rx(-1.8177656240512696) q[9];
rz(2.9986885953646967) q[9];
rx(-1.8588716944484838) q[10];
rz(-1.6373408764495425) q[10];
rx(-2.5537082414647334) q[11];
rz(2.3092663224753798) q[11];
rx(-2.259213232259093) q[12];
rz(1.9853355189817679) q[12];
rx(1.9088101422847936) q[13];
rz(1.4251731468582456) q[13];
rx(2.599308854853607) q[14];
rz(2.8714746888670666) q[14];
rx(1.8473204089992192) q[15];
rz(1.5213742765366298) q[15];
crz(-1.6986653323516463) q[2],q[1];
crz(-2.3443121668117004) q[4],q[3];
crz(1.232924110793717) q[6],q[5];
crz(0.6984347369196895) q[8],q[7];
crz(1.57484107453228) q[10],q[9];
crz(-2.8603532696581557) q[12],q[11];
crz(-0.46351375642806314) q[14],q[13];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
measure q[15] -> c[15];"""

In [37]:
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='pinq-quebec-hub/univ-toronto/default'
)

backend = service.backend(name= 'ibm_quebec')



dev = qml.device('qiskit.remote',backend=backend,wires=127)
@qml.qnode(dev)
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(NUM_QUBITS))
        variational(params, wires=range(NUM_QUBITS))

    return qml.expval(qml.PauliZ(0))


encoder = ENCODER
variational= ANSATZ

LAYERS=1
re_upload_depth=1
variational.layers = LAYERS
variational.set_wires(range(NUM_QUBITS))


num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
feat = generator.uniform(-1, 1, NUM_QUBITS)

circuit(feat,initial_parameters)

0.9476861167002012

In [41]:
circuit.qtape.to_openqasm()

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[16];\ncreg c[16];\nh q[0];\nrz(0.7709177971212526) q[0];\nh q[1];\nrz(-0.2863567564794294) q[1];\nh q[2];\nrz(0.4556280727424953) q[2];\nh q[3];\nrz(0.5459386717867756) q[3];\nh q[4];\nrz(0.17669089675761684) q[4];\nh q[5];\nrz(-0.28730772139889593) q[5];\nh q[6];\nrz(0.3881386416186652) q[6];\nh q[7];\nrz(0.23098675354619957) q[7];\nh q[8];\nrz(-0.11756691220885163) q[8];\nh q[9];\nrz(0.8542369374706567) q[9];\nh q[10];\nrz(-0.7167499738220522) q[10];\nh q[11];\nrz(0.47911148673168125) q[11];\nh q[12];\nrz(-0.16901996127639096) q[12];\nh q[13];\nrz(0.34374085032134305) q[13];\nh q[14];\nrz(-0.9307508893097289) q[14];\nh q[15];\nrz(-0.7302874499377254) q[15];\ncx q[1],q[0];\nrz(-0.2207575198959087) q[0];\ncx q[1],q[0];\ncx q[2],q[0];\nrz(0.3512517901452463) q[0];\ncx q[2],q[0];\ncx q[3],q[0];\nrz(0.42087383821716357) q[0];\ncx q[3],q[0];\ncx q[4],q[0];\nrz(0.13621415689976066) q[0];\ncx q[4],q[0];\ncx q[5],q[0];\nrz(-0.22149063567676344) q[